In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
background=None

accumilated_w=0.5

roi_top=20
roi_bottom=300
roi_right=300
roi_left=600

# 460,640



In [3]:
def calc_accumilated_w(frame,accumilated_w):
    global background
    
    if background is None:
        background=frame.copy().astype('float')
        return None
        
    cv2.accumulateWeighted( frame,background,accumilated_w)
    

def segment(frame,threshold=25):
    diff=cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    cnts,hierarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if (len(cnts)==0):
        return None
    
    else:
        handsegment=max(cnts,key=cv2.contourArea)
        return (thresholded,handsegment)

In [4]:
def count_fingers(thresholded,hand_seg):
    conv_hull=cv2.convexHull(hand_seg)
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right=tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    
    CX=(left[0]+right[0]//2)
    CY=(top[1]+bottom[1]//2)
    
    distance=pairwise.euclidean_distances([[CX,CY]],Y=[left,right,top,bottom])[0]
    
    maxdist=distance.max()
    r=int(maxdist*0.9)
    circum=(2*np.pi*r)
    
    cirular_roi=np.zeros(thresholded.shape[:2],dtype='uint8')
    
    cv2.circle(cirular_roi,(CX,CY),r,255,10)
    
    cirular_roi=cv2.bitwise_and(thresholded,thresholded,mask=cirular_roi)
    conts,hierarchy=cv2.findContours(cirular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    count=0
    
    for cnt in conts:
        (x,y,w,h)=cv2.boundingRect(cnt)
        
        outofwrist=(CY+(CY*0.25)) > (y+h)
        
        limit_points=((circum*0.25)>cnt.shape[0])
        
        if outofwrist and limit_points:
            count+=1
    return count
    

In [8]:
cam=cv2.VideoCapture(0)
numfram=0

while True:
    
    ret,frame=cam.read()
    
    frame_cop=frame.copy()
    # print(frame.shape)
    roi=frame[roi_top:roi_bottom,roi_right:roi_left ]
    
    gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    gray=cv2.GaussianBlur(gray,(7,7),0)
    
    if numfram<60:
        
        calc_accumilated_w(gray,accumilated_w)
        
        if numfram<=59:
            cv2.putText(frame_cop,"wait getting backround",(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(00,255),2)
            cv2.imshow("fingercnt",frame_cop)
    else:
        hand=segment(gray)
        
        if hand is not None:
            
            threshy,hand_sgement=hand
            
            cv2.drawContours(frame_cop,[hand_sgement+(roi_right,roi_top)],-1,(255,0,0),5)
        
            fingers=count_fingers(threshy,hand_sgement)
            
            cv2.putText(frame_cop,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('thresholded',threshy)
        
    cv2.rectangle(frame_cop,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    numfram+=1
    cv2.imshow('fingercnt',frame_cop)
    
    if cv2.waitKey(1)==ord('q'):
        break
         
cv2.destroyAllWindows()
cam.release()
        
        
    